In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [2]:
df = pd.read_csv('iris-data.csv')

In [3]:
df.head()

,sepal_length_cm,sepal_width_cm,petal_length_cm,petal_width_cm,class
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


In [4]:
df = df.dropna()
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 95 entries, 0 to 99
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   sepal_length_cm  95 non-null     float64
 1   sepal_width_cm   95 non-null     float64
 2   petal_length_cm  95 non-null     float64
 3   petal_width_cm   95 non-null     float64
 4   class            95 non-null     object 
dtypes: float64(4), object(1)
memory usage: 4.5+ KB


In [5]:
df['class'].replace(["Iris-setosa","Iris-versicolor"], [1,0], inplace=True)

In [6]:
inp_df = df.drop(df.columns[[4]], axis=1)
out_df = df.drop(df.columns[[0,1,2,3]], axis=1)

#
scaler = StandardScaler()
inp_df = scaler.fit_transform(inp_df)

# Adding an extra column to be compatible with bias
new_c = (np.zeros(shape=(inp_df.shape[0],1)) + 1)
inp_df = np.concatenate((inp_df, new_c), axis=1)

#
X_train, X_test, y_train, y_test = train_test_split(inp_df, out_df, test_size=0.2, random_state=42)

In [7]:
X_train[:5]
y_train[:5]

,class
71,0
20,1
83,0
84,0
35,1


In [8]:
def model(X, W): # W is an array of weights
    # Logistic regression model implementation
    y = np.zeros(shape=(X.shape[0], 1))
    y = np.expand_dims(y, axis=1)
    y = 1/(1+ np.exp(-np.sum(W*X)))
    
    return y

In [9]:
def loss_bce(y_true, y_pred, W):
    # Binary cross entropy loss
    loss = -(1/y_train.shape[0])*np.sum(y_true*np.log(y_pred)+(1-y_true)*np.log(1-y_pred))
    
    return loss

In [14]:
def create_mini_batches(X_train, y_train, batch_size):
    mini_batches = []
    data = np.hstack((X_train, y_train))
    np.random.shuffle(data)
    n_minibatches = data.shape[0] // batch_size
    i = 0
    for i in range(n_minibatches + 1):
        mini_batch = data[i * batch_size:(i + 1)*batch_size, :]
        X_mini = mini_batch[:, :-1]
        Y_mini = mini_batch[:, -1].reshape((-1, 1))
        mini_batches.append((X_mini, Y_mini))
    if data.shape[0] % batch_size != 0:
        mini_batch = data[i * batch_size:data.shape[0]]
        X_mini = mini_batch[:, :-1]
        Y_mini = mini_batch[:, -1].reshape((-1, 1))
        mini_batches.append((X_mini, Y_mini))
    return mini_batches

In [15]:
data = np.hstack((X_train, y_train))

In [16]:
def fit(num_weights, epoch=10, lr = 0.01):
    W = np.zeros(num_weights)
    for e in range(1, epoch+1):
        mini_batches = create_mini_batches(X_train, y_train, batch_size=16)
        for mini_batch in mini_batches:
            X_mini, y_mini = mini_batch
            y_pred = model(X_mini, W)
            print(e, ". Loss: ", loss_bce(y_train, y_pred, W), ", W: ", W)
            for i in range(W.shape[0]):
            # Calculate derivative against parameters [Implement yourself]
                x_i = X_mini[:,i]
                x_i= np.expand_dims(x_i,axis=1)
                dw_i = np.mean((y_pred-y_mini)*x_i)
            # Update parameters [Implement yourself]
                W[i] = W[i]-lr*dw_i
        
        # Plot the lines
        # Y_hat = w * X + b
        # plt.plot(X, Y_hat, label="Epoch: "+str(i))
    
    #plt.scatter(X, Y, label="dataset")
    #plt.legend()
    #plt.show()
    
    return W

In [17]:
W = fit(5)

1 . Loss:  class    0.693147
dtype: float64 , W:  [0. 0. 0. 0. 0.]
1 . Loss:  class    0.691712
dtype: float64 , W:  [ 0.00115578  0.00348421 -0.00419233 -0.00442513 -0.00125   ]
1 . Loss:  class    0.68775
dtype: float64 , W:  [-0.00061804  0.00722963 -0.00867921 -0.00876132 -0.00180174]
1 . Loss:  class    0.696916
dtype: float64 , W:  [-0.00181708  0.00920707 -0.01365889 -0.01386576 -0.00323769]
1 . Loss:  class    0.700027
dtype: float64 , W:  [-0.0030643   0.01231963 -0.01854085 -0.01904168 -0.00276859]
1 . Loss:  class    0.703695
dtype: float64 , W:  [-0.00450444  0.01611397 -0.02360138 -0.02362366 -0.00219676]
2 . Loss:  class    0.688235
dtype: float64 , W:  [-0.00595447  0.01988207 -0.02863372 -0.02818301 -0.00173339]
2 . Loss:  class    0.703733
dtype: float64 , W:  [-0.00521748  0.02402371 -0.03293267 -0.03218332 -0.00290499]
2 . Loss:  class    0.688676
dtype: float64 , W:  [-0.00661193  0.02736651 -0.03745346 -0.03714528 -0.00265101]
2 . Loss:  class    0.704923
dtype: fl